In [ ]:
import PIL
from PIL import Image, ImageEnhance, ImageDraw, ImageFont

pics = []

for i in (0.1, 0.5, 0.9):
    image=Image.open("readonly/msi_recruitment.gif")
    image=image.convert('RGB')
    r, g, b = image.split()
    r = r.point(lambda x: x * i)
    out = Image.merge('RGB', (r, g, b))
    out = out.resize((int(out.width / 2), (int(out.height / 2))))
    rect = Image.new('RGB', (out.width, 30), color = (0, 0, 0))
    d = ImageDraw.Draw(rect)
    fnt = ImageFont.truetype('readonly/fanwood-webfont.ttf', 20)
    d.text((10, 10), 'channel 0 intensity {}'.format(i), font = fnt, fill = out.getpixel((0, 50)))
    sheet = PIL.Image.new(out.mode, (out.width, out.height + rect.height))
    sheet.paste(rect, (0, out.height))
    sheet.paste(out, (0, 0))
    pics.append(sheet)

for i in (0.1, 0.5, 0.9):
    image=Image.open("readonly/msi_recruitment.gif")
    image=image.convert('RGB')
    r, g, b = image.split()
    g = g.point(lambda x: x * i)
    out = Image.merge('RGB', (r, g, b))
    out = out.resize((int(out.width / 2), (int(out.height / 2))))
    rect = Image.new('RGB', (out.width, 30), color = (0, 0, 0))
    d = ImageDraw.Draw(rect)
    fnt = ImageFont.truetype('readonly/fanwood-webfont.ttf', 20)
    d.text((10, 10), 'channel 1 intensity {}'.format(i), font = fnt, fill = out.getpixel((0, 50)))
    sheet = PIL.Image.new(out.mode, (out.width, out.height + rect.height))
    sheet.paste(rect, (0, out.height))
    sheet.paste(out, (0, 0))
    pics.append(sheet)

for i in (0.1, 0.5, 0.9):
    image=Image.open("readonly/msi_recruitment.gif")
    image=image.convert('RGB')
    r, g, b = image.split()
    b = b.point(lambda x: x * i)
    out = Image.merge('RGB', (r, g, b))
    out = out.resize((int(out.width / 2), (int(out.height / 2))))
    rect = Image.new('RGB', (out.width, 30), color = (0, 0, 0))
    d = ImageDraw.Draw(rect)
    fnt = ImageFont.truetype('readonly/fanwood-webfont.ttf', 20)
    d.text((10, 10), 'channel 2 intensity {}'.format(i), font = fnt, fill = out.getpixel((0, 50)))
    sheet = PIL.Image.new(out.mode, (out.width, out.height + rect.height))
    sheet.paste(rect, (0, out.height))
    sheet.paste(out, (0, 0))
    pics.append(sheet)
    

first_image = pics[0]

contact_sheet = PIL.Image.new(first_image.mode, (first_image.width * 3, first_image.height * 3))
x = 0
y = 0

for image in pics:
    contact_sheet.paste(image, (x, y))
    if x + first_image.width == contact_sheet.width:
        x = 0
        y = y + first_image.height 
    else:
        x = x + first_image.width

contact_sheet = contact_sheet.resize((int(contact_sheet.width/2),int(contact_sheet.height/2) ))
display(contact_sheet)

In [ ]:
import PIL
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageEnhance

# read image and convert to RGB
image=Image.open("readonly/msi_recruitment.gif")
image=image.convert('RGB')

# build a list of 9 images which have different brightnesses
#enhancer=ImageEnhance.Color(image)
rgb_values = [255 ,255, 255]
colors = []
images=[]
channel_lables = []
for i in range(3):
    for j in (0.1, 0.5, 0.9):
        temp_image = image.split()
        modified_pixels = temp_image[i].point(lambda i: i*j)
        new_rgb_values = rgb_values[:]
        new_rgb_values[i] = int(255 * j)        
        colors.append(new_rgb_values)
        temp_image[i].paste(modified_pixels)
        im = Image.merge(image.mode, temp_image)
        channel_lables.append("channel {} intensity {}".format(i, j))
        images.append(im)
        
       

# create a contact sheet from different brightnesses
first_image=images[0]
contact_sheet=PIL.Image.new(first_image.mode, (first_image.width*3,(first_image.height*3)+195))
x=0
y=0

font_file = "readonly/fanwood-webfont.ttf"
font = ImageFont.truetype(font_file,size = 60)
for i in range(0,9):
    # Lets paste the current image into the contact sheet
    contact_sheet.paste(images[i], (x, y) )
    text = ImageDraw.Draw(contact_sheet)
    text.text((x,y+first_image.height+10), channel_lables[i], fill = tuple(colors[i]), font = font)
    # Now we update our X position. If it is going to be the width of the image, then we set it to 0
    # and update Y as well to point to the next "line" of the contact sheet.
    if x+first_image.width == contact_sheet.width:
        x=0
        y=y+first_image.height+65
    else:
        x=x+first_image.width

# resize and display the contact sheet
contact_sheet = contact_sheet.resize((int(contact_sheet.width/2),int(contact_sheet.height/2) ))
display(contact_sheet)

In [ ]:
import math
import zipfile
from PIL import Image, ImageOps, ImageDraw
import pytesseract
import cv2 as cv
import numpy as np

# loading the face detection classifier
face_cascade = cv.CascadeClassifier('readonly/haarcascade_frontalface_default.xml')

# the rest is up to you!


#define a parsed source to work on:
parsed_img_src = {}

#iterate through the zip file and save all the binarized versions to parsed_img_src
with zipfile.ZipFile('readonly/images.zip', 'r') as archive:
    for entry in archive.infolist():
        with archive.open(entry) as file:
            img = Image.open(file).convert('RGB')
            parsed_img_src[entry.filename] = {'pil_img':img}
            
            
#parse all images text
for img_name in parsed_img_src.keys():
    text = pytesseract.image_to_string(parsed_img_src[img_name]['pil_img'])
    parsed_img_src[img_name]['text'] = text
    
    
#find the bounding boxes for all the faces from every page and extract them
for img_name in parsed_img_src.keys():
    open_cv_image = np.array(parsed_img_src[img_name]['pil_img']) 
    img_g = cv.cvtColor(open_cv_image, cv.COLOR_BGR2GRAY)
    faces_bounding_boxes = face_cascade.detectMultiScale(img_g, 1.3, 5)
    parsed_img_src[img_name]['faces'] = []
    for x,y,w,h in faces_bounding_boxes:
        face = parsed_img_src[img_name]['pil_img'].crop((x,y,x+w,y+h))
        parsed_img_src[img_name]['faces'].append(face)
        
        

#search the keyword in every page's text and return the faces
def search(keyword):
    for img_name in parsed_img_src:
        if (keyword in parsed_img_src[img_name]['text']):
            if(len(parsed_img_src[img_name]['faces']) != 0):
                print("Result found in file {}".format(img_name))
                h = math.ceil(len(parsed_img_src[img_name]['faces'])/5)
                contact_sheet=Image.new('RGB',(500, 100*h))
                xc = 0
                yc = 0
                for img in parsed_img_src[img_name]['faces']:
                    contact_sheet.paste(img, (xc, yc))
                    if xc + 100 == contact_sheet.width:
                        xc = 0
                        yc += 100
                    else:
                        xc += 100
                        
                display(contact_sheet)
            else:
                print("Result found in file {} \nBut there were no faces in that file\n\n".format(img_name))
    return


search('Christopher')
search('Mark')
search('pizza')

In [ ]:
import zipfile

from PIL import Image
import pytesseract
import cv2 as cv
import numpy as np

# loading the face detection classifier
face_cascade = cv.CascadeClassifier('readonly/haarcascade_frontalface_default.xml')

#other libraries
import os
import io
import numpy as np

#Function to get informationdef get_news_info(image_path):
    news_info = {}
    with zipfile.ZipFile(image_path) as zippedImgs:
        for fname in zippedImgs.namelist():
            with zippedImgs.open(fname) as imgFile:
                # read image
                cv_img = cv.imdecode(np.frombuffer(imgFile.read(), np.uint8), 1)

                # convert to grayscale
                gray = cv.cvtColor(cv_img, cv.COLOR_BGR2GRAY)

                #extract faces
                faces = face_cascade.detectMultiScale(gray, 1.3, 5)
                

  # extract text
                img = Image.open(imgFile)
                text = pytesseract.image_to_string(img)

                # store news info
                news_info[fname] = (img, text, faces)
       
    return news_info
#getting info from images
images_news_info = get_news_info('readonly/images.zip')
#getting info from small images
small_images_news_info = get_news_info('readonly/small_img.zip')
#get faces function
def get_faces(search_word, news_info):
    contact_img_height = 100
    
contact_img_width = 100

    for fname in news_info.keys():

        text = news_info[fname][1]
        if search_word in text:
            # print text
            print('Results found in file', fname)

            # check to see if there are any faces
            faces = news_info[fname][2]
            if len(faces) < 1:
                print('But there were no faces in that file!')
                continue

            # define contact sheet
            num_rows = int(np.ceil(len(faces) / 5))
            contact_sheet = Image.new('RGB', (5*contact_img_width, num_rows*contact_img_height))
            

 # extract faces
            img = news_info[fname][0]
            face_imgs = []
            for face in faces:
                cropped_img = img.crop((face[0],face[1],face[0]+face[2],face[1]+face[3]))
                face_imgs.append(cropped_img)

            # add faces to contact sheet
            x=0
            y=0

            for face_img in face_imgs:
                # resize image if larger than intended dims
                face_img_width = face_img.width
                face_img_height = face_img.height
                if (face_img_width > contact_img_width) or (face_img_height > contact_img_height):
                    
                    
contact_sheet.paste(face_img.resize((contact_img_width, contact_img_height)), (x, y) )
                else:
                    contact_sheet.paste(face_img, (x, y) )
                if x+contact_img_width == contact_sheet.width:
                    x=0
                    y=y+contact_img_height
                else:
                    x=x+contact_img_width

            display(contact_sheet)
#get faces from small images
get_faces(search_word = "Christopher", news_info = small_images_news_info)
#get faces from images
get_faces(search_word = "Mark", news_info = images_news_info)